# Data (Pre)-Processing

In [ ]:
#Importing libraries for data processing
import pandas as pd
import numpy as np

#Getting the data (fixtures, team stats, and stats against the team)
fixtures = pd.read_excel('Fixtures.xlsx').values
stats = pd.read_excel("Team Stats.xlsx").values
vs_stats = pd.read_excel("Vs Stats.xlsx").values

#Temp is used & re-used to visualize the intermediate steps
temp=[]
for i in fixtures:
    temp.append(str(i[1]).split("–"))
    
fixtures = np.delete(fixtures, 1, 1)
fixtures = np.append(fixtures, temp, 1)

fixturesArranged=[]
y = []
for i in fixtures:
    fixturesArranged.append([i[0], i[1]])
    y.append(i[2])           
    fixturesArranged.append([i[1], i[0]])
    y.append(i[3])

In [ ]:
#Creating a function that would prepare our dataset for the neural network as a tensor
def process_fixtures(fixturesArranged):
    temp=[]
    temp2=[]
    for i in fixturesArranged:
        _t1 = str(i[0]).split()
        print(_t1)
        _t2 = str(i[1]).split()
        print(_t2)
        for j in stats:
            if ((_t1[0] in str(j[0])) and (_t1[1] in str(j[0]))):
                temp.append(j)
        for j in vs_stats:
            if ((_t2[0] in str(j[0])) and (_t2[1] in str(j[0]))):
                temp2.append(j)
                
    X = np.append(fixturesArranged, temp, 1)
    X = np.append(X, temp2, 1)
    X = np.delete(X, [0,1,2], 1)
    X = np.delete(X, 21, 1)
    return X

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
def split_data():
  X = process_fixtures(fixturesArranged)

  #Splitting data into training and testing datasets
  
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

  #Scaling the data appropriately to impart uniformity to the data

  X_train = sc.fit_transform(X_train)
  X_test = sc.transform(X_test)
  return X_train, X_test, y_train, y_test

# The Model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import os

checkpoint_path = "/content/fpl.ckpt" #@param{type:"string"}
checkpoint_dir = os.path.dirname(checkpoint_path)

import tensorflow as tf
es = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=16)
ckpt = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_dir, monitor='loss', verbose=1, save_best_only=True, save_weights_only=True, mode='min')

def create_regressor():
  # Initialising the regressor
  regressor = Sequential()

  # input layer and first hidden layer
  regressor.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu', input_dim = 46))
  regressor.add(Dropout(0.2))

  # second hidden layer
  regressor.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))
  regressor.add(Dropout(0.2))

  # third hidden layer
  regressor.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu'))
  regressor.add(Dropout(0.2))

  # fourth hidden layer
  regressor.add(Dense(units = 4, kernel_initializer = 'uniform', activation = 'relu'))
  regressor.add(Dropout(0.2))

  # output layer
  regressor.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'linear'))

  # compiling the model
  regressor.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])
  
  return regressor



In [ ]:
#RUN ONLY IF TRAINING
regressor = create_regressor()
X_train, X_test, y_train, y_test = split_data()
regressor.fit(np.asarray(X_train).astype('float32'), np.asarray(y_train).astype('float32'), batch_size = 10, epochs = 10000, shuffle = True, callbacks=[es,ckpt])


['Dinamo', 'Zagreb', 'hr']
['eng', 'Chelsea']
['eng', 'Chelsea']
['Dinamo', 'Zagreb', 'hr']
['Dortmund', 'de']
['dk', 'FC', 'Copenhagen']
['dk', 'FC', 'Copenhagen']
['Dortmund', 'de']
['Celtic', 'sct']
['es', 'Real', 'Madrid']
['es', 'Real', 'Madrid']
['Celtic', 'sct']
['Benfica', 'pt']
['il', 'Maccabi', 'Haifa']
['il', 'Maccabi', 'Haifa']
['Benfica', 'pt']
['RB', 'Salzburg', 'at']
['it', 'Milan']
['it', 'Milan']
['RB', 'Salzburg', 'at']
['Sevilla', 'es']
['eng', 'Manchester', 'City']
['eng', 'Manchester', 'City']
['Sevilla', 'es']
['RB', 'Leipzig', 'de']
['ua', 'Shakhtar']
['ua', 'Shakhtar']
['RB', 'Leipzig', 'de']
['Paris', 'S-G', 'fr']
['it', 'Juventus']
['it', 'Juventus']
['Paris', 'S-G', 'fr']
['Ajax', 'nl']
['sct', 'Rangers']
['sct', 'Rangers']
['Ajax', 'nl']
['Eint', 'Frankfurt', 'de']
['pt', 'Sporting', 'CP']
['pt', 'Sporting', 'CP']
['Eint', 'Frankfurt', 'de']
['Tottenham', 'eng']
['fr', 'Marseille']
['fr', 'Marseille']
['Tottenham', 'eng']
['Napoli', 'it']
['eng', 'Liverpool'

In [ ]:
regressor.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 32)                1504      
                                                                 
 dropout_8 (Dropout)         (None, 32)                0         
                                                                 
 dense_11 (Dense)            (None, 16)                528       
                                                                 
 dropout_9 (Dropout)         (None, 16)                0         
                                                                 
 dense_12 (Dense)            (None, 8)                 136       
                                                                 
 dropout_10 (Dropout)        (None, 8)                 0         
                                                                 
 dense_13 (Dense)            (None, 4)                

# Testing Model

In [ ]:
# Not needed to run this if running from the beginning in the same session
regressor = create_regressor()
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest
regressor.load_weights(latest)

In [ ]:
# Evaluating the model


preds = regressor.predict(X_test)

y_test = np.array([int(i) for i in y_test])
print(y_test)
print(preds[:,0])

diff = preds[:,0] - y_test
print(diff)

mean = np.mean(diff)
std = np.std(diff)



print("[error_mean: {:.2f}, sample_error_stddev: {:.2f}".format(mean, std))

2/2 [==============================] - 0s 7ms/step
[0 1 1 2 4 0 0 0 7 0 0 5 0 5 1 0 1 0 0 1 2 1 0 0 2 0 1 1 3 1 1 1 0 5 0 3 4
 3 1 0 1 3 2 0 0 0 1 1 1 0]
[0.99974316 1.1807494  0.9092306  1.2957047  0.8022787  0.769771
 1.1702536  0.7166968  2.1236858  0.7612396  0.63211936 1.4589244
 1.1068571  2.6032383  0.80161184 0.7692379  0.82457376 1.5577301
 0.63580585 0.9425198  2.030761   1.9457837  0.7859644  0.63211936
 2.1906068  0.68696475 0.86391693 2.2709703  2.5386012  1.3906708
 0.75157857 0.549571   0.549571   3.9191642  1.0597888  2.6032383
 1.8246623  1.0936753  1.8532716  0.73577887 1.4416012  1.3069699
 0.7158465  0.86391693 1.4236246  1.1240792  0.6918652  0.5601651
 1.4100876  1.2434483 ]
[ 0.99974316  0.18074942 -0.09076941 -0.70429528 -3.1977213   0.76977098
  1.17025363  0.7166968  -4.87631416  0.76123959  0.63211936 -3.54107559
  1.10685706 -2.39676166 -0.19838816  0.76923788 -0.17542624  1.55773008
  0.63580585 -0.05748022  0.030761    0.94578373  0.78596443  0.63211936
  

# Predicting New Unplayed Match (City Vs Inter)

In [ ]:
fixturePred = [["Manchester City eng", "it Inter"], ["it Inter", "Manchester City eng"]] #ostrich algorithm :)
X_pred = process_fixtures(fixturePred)
X_pred = sc.transform(X_pred)
y_pred = np.reshape(regressor.predict(X_pred), (1,-1))[0]

['Manchester', 'City', 'eng']
['it', 'Inter']
['it', 'Inter']
['Manchester', 'City', 'eng']
1/1 [==============================] - 0s 28ms/step


In [ ]:
print(y_pred)

[1.5413448 0.8568226]


In [ ]:
import math
from scipy.stats import norm
z_score = ((y_pred[0]-y_pred[1])-mean)/math.sqrt(2*std)
p_value = norm.cdf(z_score) #probability that City wins given that errors made by the regressor are independent
print(p_value)

0.6748977586759732
